In [18]:
import requests
import re
import json
import numpy as np
import pandas as pd
import datetime
import time
from pyecharts import options as opts
from pyecharts.charts import Map
from pyecharts.faker import Faker
from pyecharts.charts import Geo,Pie
from pyecharts.globals import ChartType
from pyecharts.charts import *
from pyecharts import options as opts
from pyecharts.globals import ChartType
from pyecharts.charts import Line
from pyecharts.components import Table
from pyecharts.options import ComponentTitleOpts

In [19]:
headers = {"User-Agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'}

In [20]:
# You could change the holds of your portolio at this position. 
stock_code=['AAPL','TSLA','BX','MSFT','V','BAC','MS','GS']
index_code = ['^GSPC','^DJI']
quantity=[100,100,100,100,100,100,100,100]
purchase_date=[20221231,20221231,20221231,20221231,20221231,20221231,20221231,20221231]
purchase_price=[0,0,0,0,0,0,0,0]

In [21]:
# There We don't save the data, we just use the requests to receive the data.
def get_data(stock_code,quantity,purchase_date,purchase_price=[0,0,0,0]):
    data_total = pd.DataFrame(pd.date_range(datetime.date.today()-datetime.timedelta(365),datetime.date.today())).rename(columns = {0:'date'})
    data_total['date'] = data_total['date'].apply(lambda x:x.strftime('%Y%m%d'))
    for i in range(len(stock_code)):
        link = 'https://query1.finance.yahoo.com/v8/finance/chart/'+stock_code[i]+'?region=US&lang=en-US&includePrePost=false&interval=1d&useYfid=true&range=1y&corsDomain=finance.yahoo.com&.tsrc=finance'
        response = requests.get(link,headers = headers)
        data = response.text
        data = json.loads(data)
        data_cleaned = pd.DataFrame(data['chart']['result'][0][ 'indicators']['adjclose'][0]['adjclose']
                                    ,columns = ['adjclose_'+stock_code[i]+'_price'],index = list(map(lambda x:time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(x)),data['chart']['result'][0]['timestamp'])))
        data_cleaned.index = pd.to_datetime(data_cleaned.index)
        data_cleaned = data_cleaned.reset_index()
        data_cleaned = data_cleaned.rename(columns = {'index':'date'})
        data_cleaned['date'] = data_cleaned['date'].apply(lambda x:x.strftime('%Y%m%d'))
        data_cleaned = data_cleaned[data_cleaned['date'].astype(int)>=purchase_date[i]]
        if purchase_price[i] != 0:
            data_cleaned['adjclose_'+stock_code[i]+'_price'].iloc[0] = purchase_price[i] 

        data_cleaned['total_values_'+stock_code[i]] = data_cleaned['adjclose_'+stock_code[i]+'_price']*quantity[i]

        data_total = pd.merge(data_total,data_cleaned, how = 'left',on = 'date')
    data_total = data_total[data_total.T.isna().sum()!=8]
    data_total.index = data_total['date']
    data_total = data_total.drop('date',axis = 1)
    data_total.index = pd.to_datetime(data_total.index.astype(str))
    return  data_total
def get_1d_index(index):
    data_total= pd.DataFrame()
    for i in range(len(index)):
        link = 'https://query1.finance.yahoo.com/v8/finance/chart/'+index[i]+'?region=US&lang=en-US&includePrePost=false&interval=2m&useYfid=true&range=1d&corsDomain=finance.yahoo.com&.tsrc=finance'

        response = requests.get(link,headers = headers)
        data = response.text
        data = json.loads(data)
        data_cleaned = pd.DataFrame(data['chart']['result'][0][ 'indicators']['quote'][0]['close']
                                ,columns = [index[i]],index = list(map(lambda x:time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(x)),data['chart']['result'][0]['timestamp'])))
        data_cleaned.index = pd.to_datetime(data_cleaned.index)
        data_total = data_total.append(data_cleaned.T)
    data_total = data_total.T  
    return data_total
def get_5d_data(stock_code,quantity,purchase_date,purchase_price=[0,0,0,0]):
    data_total= pd.DataFrame()
    for i in range(len(stock_code)):
        link = 'https://query1.finance.yahoo.com/v8/finance/chart/'+stock_code[i]+'?region=US&lang=en-US&includePrePost=false&interval=15m&useYfid=true&range=5d&corsDomain=finance.yahoo.com&.tsrc=finance'
        response = requests.get(link,headers = headers)
        data = response.text
        data = json.loads(data)
        data_cleaned = pd.DataFrame(data['chart']['result'][0][ 'indicators']['quote'][0]['close']
                                ,columns = ['adjclose_'+stock_code[i]+'_price'],index = list(map(lambda x:time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(x)),data['chart']['result'][0]['timestamp'])))
        data_cleaned.index = pd.to_datetime(data_cleaned.index)
        data_cleaned['total_values_'+stock_code[i]] = data_cleaned['adjclose_'+stock_code[i]+'_price']*quantity[i]
        data_total = data_total.append(data_cleaned.T)
    data_total = data_total.T  
    return data_total
def get_1d_data(stock_code,quantity,purchase_date,purchase_price=[0,0,0,0]):
    data_total= pd.DataFrame()
    for i in range(len(stock_code)):
        link = 'https://query1.finance.yahoo.com/v8/finance/chart/'+stock_code[i]+'?region=US&lang=en-US&includePrePost=false&interval=2m&useYfid=true&range=1d&corsDomain=finance.yahoo.com&.tsrc=finance'

        response = requests.get(link,headers = headers)
        data = response.text
        data = json.loads(data)
        data_cleaned = pd.DataFrame(data['chart']['result'][0][ 'indicators']['quote'][0]['close']
                                ,columns = ['adjclose_'+stock_code[i]+'_price'],index = list(map(lambda x:time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(x)),data['chart']['result'][0]['timestamp'])))
        data_cleaned.index = pd.to_datetime(data_cleaned.index)
        data_cleaned['total_values_'+stock_code[i]] = data_cleaned['adjclose_'+stock_code[i]+'_price']*quantity[i]
        data_total = data_total.append(data_cleaned.T)
    data_total = data_total.T  
    return data_total
def get_1y_index(index):
    data_total= pd.DataFrame()
    for i in range(len(index)):
        link = 'https://query1.finance.yahoo.com/v8/finance/chart/'+stock_code[i]+'?region=US&lang=en-US&includePrePost=false&interval=1d&useYfid=true&range=1y&corsDomain=finance.yahoo.com&.tsrc=finance'

        response = requests.get(link,headers = headers)
        data = response.text
        data = json.loads(data)
        data_cleaned = pd.DataFrame(data['chart']['result'][0][ 'indicators']['quote'][0]['close']
                                ,columns = [index[i]],index = list(map(lambda x:time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(x)),data['chart']['result'][0]['timestamp'])))
        data_cleaned.index = pd.to_datetime(data_cleaned.index)
        data_total = data_total.append(data_cleaned.T)
    data_total = data_total.T  
    return data_total
def get_nav(data):
    result = pd.DataFrame()
    data = data.dropna()
    for i in data.columns:
        if 'total_valu' in i:
            result = result.append(pd.DataFrame(data[i]).T)
    result = result.sum().T
    result = pd.DataFrame(result).reset_index().rename(columns = {'index':'time',0:'nav','date':'time'})
    return result

    



Return: First grahp is about the NAV of fund and the benchmark. 

In [27]:
index_data = get_1y_index(index_code)
index_data = index_data.reset_index().rename(columns = {
    'index':'time'
})
index_data['time'] = index_data['time'].apply(lambda x:x.strftime('%Y%m%d'))
data_for_p_or = get_data(stock_code,quantity,purchase_date,purchase_price=[0,0,0,0,0,0,0,0])
data_for_p = get_nav(data_for_p_or)
data_for_p['time'] = data_for_p['time'].apply(lambda x:x.strftime('%Y%m%d'))
data_total_for_draw = pd.merge(data_for_p,index_data,on = 'time',how = 'left')
data_total_for_draw = data_total_for_draw.set_index('time')
data_total_for_draw = data_total_for_draw/data_total_for_draw.iloc[0]
data_total_for_draw=data_total_for_draw.reset_index()
data_total_for_draw['time'] = pd.to_datetime(data_total_for_draw['time'])
values_list = list(data_total_for_draw.columns)
values_list.remove('time')
return_of_p = Line()
x=data_total_for_draw['time'].to_list()
return_of_p .add_xaxis(x)
for i in values_list:
    return_of_p .add_yaxis(series_name=i, y_axis=(data_total_for_draw[i]-1).to_list(), \
            is_symbol_show=False, is_smooth=True, 
            )  # is_symbol_show显示点图例文本
index_list = list(data_total_for_draw.columns)
index_list.remove('time')
index_list.remove('nav')
for i in index_list:
    data_total_for_draw['Relative_reture_to_'+i] = data_total_for_draw[i]*data_total_for_draw['nav']-1
    return_of_p .add_yaxis(series_name='Relative_reture_to_'+i, y_axis=data_total_for_draw['Relative_reture_to_'+i].to_list(), \
            is_symbol_show=False, is_smooth=True, areastyle_opts=opts.AreaStyleOpts(opacity=0.5)
            )

return_of_p .set_global_opts(
    legend_opts=opts.LegendOpts(pos_left="left"),
    datazoom_opts=[
        opts.DataZoomOpts(range_start=0, range_end=100),
        opts.DataZoomOpts(type_="inside", range_start=None, range_end=None),
    ],  # 显示缩放
    xaxis_opts=opts.AxisOpts(
        min_=x[0],max_=x[-1],
        type_='time',
        name='日期'
    ),  # x轴设置
    yaxis_opts=opts.AxisOpts(
                 min_=-0.5, max_=1,
        type_='value',
        name='nav_pct_change'
    ),  # y轴设置
    title_opts=opts.TitleOpts(title='return', pos_top='30px', pos_left='center'),  # 居中图例名称
    tooltip_opts=opts.TooltipOpts(trigger="axis", axis_pointer_type="cross"), ) # 点击到点

# return_of_p .render()
# You could run this commond to only see this picture.

'C:\\Users\\86188\\Desktop\\render.html'

Sharpe

In [28]:
def timeline(chart_list, time_list):
    '''
    绘制时间线
    '''
    tl = Timeline()
    for i in range(len(time_list)):
        tl.add(chart_list[i], time_list[i])
    return tl
data_colection = []
for i in range(2,len(data_total_for_draw)):
    data_for_table = data_total_for_draw.iloc[0:i]
    return1 = str((data_for_table['nav'].iloc[i-1]-data_for_table['nav'].iloc[0])*100)[:4]
    MxD = ((data_total_for_draw['nav'].cummax()-data_total_for_draw['nav'])/data_total_for_draw['nav'].cummax()).max()
    MxD = str(MxD*100)[:4]
    mean = data_for_table['nav'].pct_change().mean()*len(data_for_table['nav'].pct_change())-0.03/250
    std = data_for_table['nav'].pct_change().std()*np.sqrt(len(data_for_table['nav'].pct_change()))
    sharpe = str(mean/std)[:4]
    AV = str(data_for_table['nav'].pct_change().std()*100)[:4]
    Bins = ['Return(%)','Maximum Drawdown(%)','Sharpe','annualized volatility']
    x = [data_for_table['time'].apply(lambda x:x.strftime('%Y-%m-%d'))[i-1],float(return1)
         ,float(MxD)
         ,float(sharpe),float(AV)]
    data_colection.append(x)
data_colection=pd.DataFrame(data_colection[1:])
data_colection.columns = ['time','Return(%)','Maximum Drawdown(%)','Sharpe','annualized volatility']
bars = list()
dates = list()
values_list = list(data_colection.columns)
values_list.remove('time')
for i in values_list:
    bar = (
            Bar()
            .add_xaxis(data_colection['time'].to_list())
            .add_yaxis(i,data_colection[i].to_list())
            .set_series_opts(
            label_opts=opts.LabelOpts(is_show=False, color="black",position="outside", font_size=10))
        .set_global_opts(datazoom_opts=opts.DataZoomOpts(orient='horizontal',pos_top = 'top'),
                         legend_opts=opts.LegendOpts(type_="scroll", pos_left="right", orient="vertical"),
                        title_opts=opts.TitleOpts(title=i,pos_left = 'left', pos_top = 'top'),
            tooltip_opts=opts.TooltipOpts(is_show=True)
            )
        
    
    
    
    )
    bars.append(bar)
    dates.append(i)
tl = timeline(bars,dates)
# tl.render("test.html")
# You could run this commond to only see this picture.

'C:\\Users\\86188\\Desktop\\test.html'

In [29]:
data_for_gaude = data_colection.set_index('time').iloc[-1]
gauge_list = list()
dates = list()
for i in range(len(data_for_gaude.values)):
    
    gauge = (Gauge()
              .add(data_for_gaude.index[i], [(data_for_gaude.index[i],data_for_gaude.values[i])])
              )
    dates.append(data_for_gaude.index[i])
    gauge_list.append(gauge)
    gauge.render(data_for_gaude.index[i]+'.html')
gauge_total = timeline(gauge_list,dates)
# gauge_total.render("gauge_total.html")


'C:\\Users\\86188\\Desktop\\gauge_total.html'

distribution

In [40]:
temp = data_for_p_or.T.reset_index()
temp = temp[temp['index'].str.contains('adjclose')].set_index('index').T
temp=(temp.pct_change().dropna()*100).T
bars = list()
dates = list()
for j in range(len(temp.columns)):
    temp_1 = pd.DataFrame(temp[temp.columns[j]])
    temp_1=temp_1.reset_index()
    temp_1['index'] =temp_1['index'].apply(lambda x:re.findall('_(.*?)_',x,re.S)[0])
    temp_1['color'] = temp_1[temp_1.columns[1]].apply(lambda x:'red' if x <0 else 'green')
    data_trans = []
    bar = (
        Bar()
        .add_xaxis(temp_1['index'].to_list())
        .add_yaxis(str(temp.columns[j])[:10],temp_1[temp_1.columns[1]].to_list())
         .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
    )
    bars.append(bar)
    dates.append(str(temp.columns[j])[:10])
    
bar = timeline(bars,dates)
# bar.render("bartest.html")

'C:\\Users\\86188\\Desktop\\bartest.html'

In [31]:
temp = data_for_p_or.T.reset_index()
temp = temp[temp['index'].str.contains('adjclose')].set_index('index').T
temp=(temp.pct_change().dropna()*100).T
pies = list()
dates = list()
for j in range(len(temp.columns)):
    temp_1 = pd.DataFrame(temp[temp.columns[j]])
    temp_1=temp_1.reset_index()
    temp_1['index'] =temp_1['index'].apply(lambda x:re.findall('_(.*?)_',x,re.S)[0])
    temp_1['color'] = temp_1[temp_1.columns[1]].apply(lambda x:'red' if x <0 else 'green')
    temp_1[temp_1.columns[1]] = temp_1[temp_1.columns[1]].apply(lambda x:-x if x <0 else x)
    data_trans = []
    for i in range(len(temp_1)):
        data_trans.append([temp_1['index'].iloc[i],temp_1[temp_1.columns[1]].iloc[i]])
    p = (
        Pie()
        .add('',data_trans,rosetype="radius")
        .set_colors(temp_1['color'].to_list())
    )
    pies.append(p)
    dates.append(str(temp.columns[j])[:10])
    
pie = timeline(pies,dates)
# pie.render("pietest.html")

'C:\\Users\\86188\\Desktop\\pietest.html'

oneday,fiveday,1month,6month,YTD,1y

In [32]:
data1d = get_nav(get_1d_data(stock_code,quantity,purchase_date,purchase_price))
index1d = get_1d_index(index=index_code)
data1d = data1d.set_index('time')
data1d =  data1d/data1d.iloc[0]
index1d = index1d/index1d.iloc[0]
oneday = Line()
x=data1d.reset_index()['time'].to_list()
oneday.add_xaxis(x)
for i in data1d.columns:
    oneday.add_yaxis(series_name=i, y_axis=(data1d[i]-1).to_list(), \
                is_symbol_show=False, is_smooth=True, 
                )
for i in index1d.columns:
    oneday.add_yaxis(series_name=i, y_axis=(index1d[i]-1).to_list(), \
                is_symbol_show=False, is_smooth=True, 
                )
oneday.set_series_opts(
            label_opts=opts.LabelOpts(is_show=False, color="black",position="outside", font_size=10))
oneday.set_global_opts(datazoom_opts=opts.DataZoomOpts(orient='horizontal',pos_top = 'top'),
                         legend_opts=opts.LegendOpts(type_="scroll", pos_left="right", orient="vertical"),
                        title_opts=opts.TitleOpts(title='Return',pos_left = 'left', pos_top = 'top'),
           tooltip_opts=opts.TooltipOpts(is_show=True,trigger="axis", axis_pointer_type="cross")
            )
# oneday.render()

'C:\\Users\\86188\\Desktop\\render.html'

In [33]:
data_copy = data_total_for_draw.copy()
if len(data_copy) <= 5:
    data5d = data_copy
else:
    data5d = data_total_for_draw.iloc[-5:].copy()
data5d=data5d.set_index('time')

fiveday = Line()
x=data5d.reset_index()['time'].to_list()
fiveday.add_xaxis(x)
for i in data5d.columns:
    fiveday.add_yaxis(series_name=i, y_axis=(data5d[i]-1).to_list(), \
                is_symbol_show=False, is_smooth=True, 
                )
fiveday.set_series_opts(
            label_opts=opts.LabelOpts(is_show=False, color="black",position="outside", font_size=10))
fiveday.set_global_opts(datazoom_opts=opts.DataZoomOpts(orient='horizontal',pos_top = 'top'),
                         legend_opts=opts.LegendOpts(type_="scroll", pos_left="right", orient="vertical"),
                        title_opts=opts.TitleOpts(title='Return',pos_left = 'left', pos_top = 'top'),
            tooltip_opts=opts.TooltipOpts(is_show=True,trigger="axis", axis_pointer_type="cross")
            )
# fiveday.render()

'C:\\Users\\86188\\Desktop\\render.html'

In [34]:
data_copy = data_total_for_draw.copy()
data1m = data_copy[data_copy['time'].apply(lambda x:x.strftime('%Y%m%d')).astype(int) >= int((datetime.date.today()-datetime.timedelta(30)).strftime('%Y%m%d'))]
data1m=data1m.set_index('time')

month = Line()
x=data1m.reset_index()['time'].to_list()
month.add_xaxis(x)
for i in data1m.columns:
    month.add_yaxis(series_name=i, y_axis=(data1m[i]-1).to_list(), \
                is_symbol_show=False, is_smooth=True, 
                )
month.set_series_opts(
            label_opts=opts.LabelOpts(is_show=False, color="black",position="outside", font_size=10))
month.set_global_opts(datazoom_opts=opts.DataZoomOpts(orient='horizontal',pos_top = 'top'),
                         legend_opts=opts.LegendOpts(type_="scroll", pos_left="right", orient="vertical"),
                        title_opts=opts.TitleOpts(title='Return',pos_left = 'left', pos_top = 'top'),
            tooltip_opts=opts.TooltipOpts(is_show=True,trigger="axis", axis_pointer_type="cross")
            )
# month.render()

'C:\\Users\\86188\\Desktop\\render.html'

In [35]:
data_copy = data_total_for_draw.copy()
data3m = data_copy[data_copy['time'].apply(lambda x:x.strftime('%Y%m%d')).astype(int) >= int((datetime.date.today()-datetime.timedelta(90)).strftime('%Y%m%d'))]
data3m=data3m.set_index('time')

month3 = Line()
x=data3m.reset_index()['time'].to_list()
month3.add_xaxis(x)
for i in data1m.columns:
    month3.add_yaxis(series_name=i, y_axis=(data3m[i]-1).to_list(), \
                is_symbol_show=False, is_smooth=True, 
                )
month3.set_series_opts(
            label_opts=opts.LabelOpts(is_show=False, color="black",position="outside", font_size=10))
month3.set_global_opts(datazoom_opts=opts.DataZoomOpts(orient='horizontal',pos_top = 'top'),
                         legend_opts=opts.LegendOpts(type_="scroll", pos_left="right", orient="vertical"),
                        title_opts=opts.TitleOpts(title='Return',pos_left = 'left', pos_top = 'top'),
            tooltip_opts=opts.TooltipOpts(is_show=True,trigger="axis", axis_pointer_type="cross")
            )
# month3.render()

'C:\\Users\\86188\\Desktop\\render.html'

In [36]:
data_copy = data_total_for_draw.copy()
data6m = data_copy[data_copy['time'].apply(lambda x:x.strftime('%Y%m%d')).astype(int) >= int((datetime.date.today()-datetime.timedelta(180)).strftime('%Y%m%d'))]
data6m=data6m.set_index('time')

month6 = Line()
x=data6m.reset_index()['time'].to_list()
month6.add_xaxis(x)
for i in data1m.columns:
    month6.add_yaxis(series_name=i, y_axis=(data6m[i]-1).to_list(), \
                is_symbol_show=False, is_smooth=True, 
                )
month6.set_series_opts(
            label_opts=opts.LabelOpts(is_show=False, color="black",position="outside", font_size=10))
month6.set_global_opts(datazoom_opts=opts.DataZoomOpts(orient='horizontal',pos_top = 'top'),
                         legend_opts=opts.LegendOpts(type_="scroll", pos_left="right", orient="vertical"),
                        title_opts=opts.TitleOpts(title='Return',pos_left = 'left', pos_top = 'top'),
            tooltip_opts=opts.TooltipOpts(is_show=True,trigger="axis", axis_pointer_type="cross")
            )
# month6.render()

'C:\\Users\\86188\\Desktop\\render.html'

In [37]:
data_copy = data_total_for_draw.copy()
data1y = data_copy[data_copy['time'].apply(lambda x:x.strftime('%Y%m%d')).astype(int) >=int(datetime.date.today().strftime('%Y'))*10000]
data1y=data1y.set_index('time')

ytd = Line()
x=data1y.reset_index()['time'].to_list()
ytd.add_xaxis(x)
for i in data1m.columns:
    ytd.add_yaxis(series_name=i, y_axis=(data1y[i]-1).to_list(), \
                is_symbol_show=False, is_smooth=True, 
                )
ytd.set_series_opts(
            label_opts=opts.LabelOpts(is_show=False, color="black",position="outside", font_size=10))
ytd.set_global_opts(datazoom_opts=opts.DataZoomOpts(orient='horizontal',pos_top = 'top'),
                         legend_opts=opts.LegendOpts(type_="scroll", pos_left="right", orient="vertical"),
                        title_opts=opts.TitleOpts(title='Return',pos_left = 'left', pos_top = 'top'),
            tooltip_opts=opts.TooltipOpts(is_show=True,trigger="axis", axis_pointer_type="cross")
            )
# ytd.render()

'C:\\Users\\86188\\Desktop\\render.html'

In [38]:
charts = [ytd,month6,month3,month,fiveday,oneday]
dates = ['1Y','6M','3M','5D','1D']
chart_total = timeline(charts,dates)
# chart_total.render("chart_total.html")

'C:\\Users\\86188\\Desktop\\chart_total.html'

THis is our final step-- you could just choose what your panel looks like. 

In [42]:
page = Page(
    layout=Page.DraggablePageLayout,
)
page.add(return_of_p,tl,gauge_total,bar,pie,chart_total)
page.render('大屏.html')

'C:\\Users\\86188\\Desktop\\大屏.html'

In [44]:
Page.save_resize_html("大屏.html", cfg_file="chart_config.json", dest="大屏最终效果.html")

'<!DOCTYPE html>\n<html>\n<head>\n    <meta charset="UTF-8">\n    <title>Awesome-pyecharts</title>\n                <script type="text/javascript" src="https://assets.pyecharts.org/assets/v5/echarts.min.js"></script>\n            <script type="text/javascript" src="https://assets.pyecharts.org/assets/v5/jquery.min.js"></script>\n            <script type="text/javascript" src="https://assets.pyecharts.org/assets/v5/jquery-ui.min.js"></script>\n            <script type="text/javascript" src="https://assets.pyecharts.org/assets/v5/ResizeSensor.js"></script>\n\n            <link rel="stylesheet"  href="https://assets.pyecharts.org/assets/v5/jquery-ui.css">\n\n</head>\n<body >\n    <style>.box {  } </style>\n        \n    <div class="box">\n                <div id="3fcae56e1052441297088c17757ce8a0" class="chart-container" style="position: absolute; width: 900px; height: 395px; top: 33.5px; left: 9px;"></div>\n    <script>\n        var chart_3fcae56e1052441297088c17757ce8a0 = echarts.init(\n